# A lőcsei szászok alkalmi nyomtatványai

Első lépésként a pandas könyvtár segítségével beolvasom az elkészült Excel fájlokat. 

In [1]:
import pandas
import networkx as nx
from pyvis import network
import matplotlib.pyplot as plt
import math
import textwrap
import warnings
warnings.filterwarnings('ignore')

records = pandas.read_excel('kiadvanyok.xlsx', sheet_name='lőcsei vonatkozás')
records.columns = records.columns.str.strip()
records.set_index('RMNy',  inplace=True)
records.head()

,Unnamed: 0,év,típusa,esemény,a nyomtatvány rövid leírása,a nyomtatvány címzettjei,szerzők és a nyomtatványban említett kapcsolódó nevek
RMNy,,,,,,,
1076,4,1614,Gyászversek,halál,Thurzó Kristóf főispán felett mondott gyászversek,Thurzó Kristóf,"Holtzmann, Stephanus"
1230,18,1620,Oktató versek,beiktatás,A versek a lőcsei városi magistrátus és az ala...,"Cramer, Caspar, id.","Musculus, Wolfgang"
1231,19,1620,Gyászbeszéd,halál,Stephanus Holtzmann (Xylander) szepesváraljai ...,"Holtzmann, Stephanus","Zabeler, Peter; Horlerus, Stanislaus"
1254,20,1621,Lakodalmi ének,esküvő,Daniel Cramer és Christina Wolf 1621. szeptemb...,"Cramer, Daniel; Wolf, Christina","Musculus, Wolfgang; Wolf, Johann"
1255,21,1621,Oktató versek,általános üdvözlés,Tizenkét vers az elöljáróknak szükséges tizenk...,NaN,"Musculus, Wolfgang"


In [2]:
citizens = pandas.read_excel('szemelyek.xlsx')
citizens.head()

,id,név,születési hely,névváltozat,foglalkozás,életrajzi hivatkozás,rövid leírás,RMNy (alkalmi)
0,1,[szervezet] a besztercebányai evangélikus isko...,NaN,NaN,vizsga,NaN,A tételeket 1642 májusában vitatták meg a besz...,1934
1,2,[szervezet] a lőcsei evangélikus gimnázium elő...,NaN,NaN,program,NaN,Meghívó a lőcsei evangélikus gimnázium hallgat...,2223
2,3,[szervezet] a lőcsei evangélikus gimnázium ünn...,NaN,NaN,program,NaN,A lőcsei evangélikus gimnázium 1666. január 30...,3295
3,4,[szervezet] a lőcsei evangélikus gimnázium viz...,NaN,NaN,vizsga,NaN,A nyomtatvány a lőcsei evangélikus gimnázium 1...,3296
4,5,[szervezet] a lőcsei evangélikus gimnázium viz...,NaN,NaN,vizsga,NaN,1646. szeptember 9. A tételeket szeptember 25-...,2147


Az alábbiakban a beolvasott nyomtatványok "címzettek" és "szerzők" oszlopait átolvasva, a nevekhez a polgár-adatbázisból hozzárendeljük a megfelelő polgárt, és egy relációs táblát készítünk a kettőből:

In [3]:
def get_from_citizen_list(name, record):
    
    name=name.replace('[', '')
    name=name.replace(']', '')
    name=name.replace('-', '')
    
    index_name = citizens['név'].str.contains(name, na=False)
    index_alt_name = citizens['névváltozat'].str.contains(name, na=False)
    
    index_list = citizens[index_name]
    index_alt_list = citizens[index_alt_name]
    
    filtered_by_record = index_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    filtered_by_record_alt = index_alt_list['RMNy (alkalmi)'].astype(str).str.contains(str(record))
    if not filtered_by_record.empty:
        return filtered_by_record.index[0]
    if not filtered_by_record_alt.empty:
        return filtered_by_record_alt.index[0]
    return math.nan


relations = []
counter = { }

for index, record in records.iterrows():
    recipient_names = record['a nyomtatvány címzettjei']
    sender_names = record['szerzők és a nyomtatványban említett kapcsolódó nevek']
    
    if isinstance(recipient_names, str) and isinstance(sender_names, str):
        recipients = recipient_names.split(';') 
        senders = sender_names.split(';')
        for recipient in recipients:
            target_id = get_from_citizen_list(recipient, index)
            
            for sender in senders:
                sender_id = get_from_citizen_list(sender, index)
                relation = {}
                relation['record_id'] = index
                relation['sender_name'] = sender
                if not math.isnan(sender_id):
                    name = str(citizens.iloc[sender_id]['név'])
                    relation['sender_id'] = int(sender_id)
                    relation['sender_name'] = name
                    if name not in counter:
                        counter[name] = {
                            'sender': 0,
                            'target': 0
                        }
                    counter[name]['sender'] += 1
                else:
                    relation['sender_id'] = -1
                
                relation['target_name'] = recipient
                if not math.isnan(target_id):
                    name = str(citizens.iloc[target_id]['név'])
                    relation['target_id'] = int(target_id)
                    relation['target_name'] = name
                    
                    if name not in counter:
                        counter[name] = {
                            'sender': 0,
                            'target': 0
                        }
                    counter[name]['target'] += 1
                    
                else:
                    relation['target_id'] = -1
                    
                
                relations.append(relation)

                

df = pandas.DataFrame.from_dict(counter).transpose()
df['sum'] = df['sender']+df['target']
df = df.sort_values(by=['sum'], ascending=False)
df = df.style.set_caption("A nyomtatványok között leggyakrabban előforduló nevek")
df

,sender,target,sum
"Seelmann, Christian (1626-1675/76)",10,92,102
"Zabeler, Hiob (1628–1664)",2,91,93
"Böhm, Christoph (1626-1660)",0,67,67
"Zauchtler, Anna Catharina (1635-1668)",0,65,65
"Hain, Caspar (1632-1687)",0,50,50
"Schlegel, Christoph (1613-1678)",2,48,50
"Röser, Jakob (1641-1689)",0,43,43
"Kemmel, Johann (1636-1685)",4,32,36
"Windisch, Johann (1605-1672)",2,29,31
"Frölich, David (1595-1648)",0,25,25


In [4]:
relations_df = pandas.DataFrame.from_records(relations)
relations_df.head()


,record_id,sender_name,sender_id,target_name,target_id
0,1076,"Holtzmann, Stephanus (1572-1619)",220,Thurzó Kristóf (?-1614),548
1,1230,"Musculus, Wolfgang (?-1638)",360,"Cramer, Caspar, id.",-1
2,1231,"Zabeler, Peter id. (1578-1645)",593,"Holtzmann, Stephanus (1572-1619)",220
3,1231,"Horlerus, Stanislaus",-1,"Holtzmann, Stephanus (1572-1619)",220
4,1254,"Musculus, Wolfgang (?-1638)",360,"Cramer, Daniel (? -?)",78


Az alábbiakban a relációs táblából a networkx könyvtár segítségével relációs gráfot készítek.

In [5]:
G = nx.from_pandas_edgelist(relations_df, source='sender_name', target='target_name', edge_attr=True, create_using=nx.MultiDiGraph())

Az alábbiakban meghatározzuk az eseménytípusok színkódjait.<br>
 <span style="color: black"><strong> Halál - fekete</strong></span><br>
 <span style="color: purple"><strong>Esküvő - lila</strong></strong></span><br>
 <span style="color: red"><strong>Beiktatás - piros</strong></span><br>
 <span style="color: blue"><strong></strong>Általános üdvözlés - kék</strong></strong></strong></strong></span><br>
 <span style="color: lightgrey"><strong>Alapértelmezett - szürke</strong></strong></strong></span><br>
 

In [6]:
edge_colors = {
    'halál': 'black',
    'esküvő': 'purple',
    'beiktatás': 'red',
    'általános üdvözlés': 'blue',
    'default': 'grey'
}

def get_edge_color(type):
    if type in edge_colors.keys():
        return edge_colors[type]
    else:
        return edge_colors['default']


A generált személyhez hozzáadjuk a rövid leírást, ami a pontokra való kattintáskor megjelenít. Illetve minden relációhoz hozzárendeljük a nyomtatvány számát és rövid leírását, majd megjelenítjük

In [7]:
net = network.Network(notebook=True, directed=True)
net.from_nx(G)

for edge in net.edges:
    to_node = net.get_node(edge['to'])
    from_node = net.get_node(edge['from'])
    to_node['title'] = "Nem található leírás."
    from_node['title'] = "Nem található leírás."
    
    record_id = edge['record_id']
    
    edge['title'] = str(record_id) + '\n' + textwrap.fill(records.loc[record_id]['a nyomtatvány rövid leírása']) 
    edge['color'] = get_edge_color(records.loc[record_id]['esemény'])
    
    if edge['sender_id'] != -1:
        string_sender = citizens.iloc[edge['sender_id']]['rövid leírás']
        if isinstance(string_sender, str):
            from_node['title'] = str(edge['from']) + '\n' + textwrap.fill(string_sender, 40)
    if edge['target_id'] != -1:
        string_target = citizens.iloc[edge['target_id']]['rövid leírás']
        if isinstance(string_target, str):
            to_node['title'] = str(edge['to']) + '\n' + textwrap.fill(string_target, 40)
    
        
net.toggle_physics(True)
net.show('relationship.html')

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
